# Business Understanding
We would like to track Corona VIRUS spread across countries and with personal local information

The general information is not so relevant for me I would like to have a deep dive local development of the spread



# Goals
* We would like to understand the data quality
* Everything should be automated as much as possible: how many clicks do we need to execute the full pipeline

# Constraints:
Each notebook should be left clean and ready for full execution.

# Data Understanding
* Robert Koch Institue (webscraping) https://www.rki.de/DE/Content?InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
* John Hopkins (github access) https://github.com/CSSEGISandData/COVID-19.git
* REST API services to retrieve data https://npgeo-corona-npgeo-de.hub.arcgis.com/

In [618]:
import subprocess
import os

import pandas as pd

import requests

from bs4 import BeautifulSoup

import json

pd.set_option('display.max_rows', 500)

To be done in terminal
* cd /Data_Science_COVID-19/data/raw
* git clone https://github.com/CSSEGISandData/COVID-19.git

In [619]:
#subprocess library accesses the terminal in python. PIPE means pipeline for standard out and error
git_pull = subprocess.Popen("/usr/bin/git pull", cwd = os.path.dirname('../data/raw/COVID-19/'), shell = True, stdout= subprocess.PIPE, stderr = subprocess.PIPE)

(out, error) = git_pull.communicate()

print('Error: ' + str(error))

print('out: ' + str(out))

Error: b''
out: b'Already up to date.\n'


In [620]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

In [621]:
pd_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/17/22,6/18/22,6/19/22,6/20/22,6/21/22,6/22/22,6/23/22,6/24/22,6/25/22,6/26/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,181534,181574,181666,181725,181808,181912,181987,182033,182072,182149
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,277141,277141,277409,277444,277663,277940,278211,278504,278793,279077
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,265968,265971,265975,265985,265993,266006,266015,266025,266030,266038
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,43449,43449,43449,43449,43449,43774,43774,43774,43774,43774
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,99761,99761,99761,99761,99761,99761,99761,99761,99761,99761


# Webscrapping
* visit the robert koch institute website
* use an web-browser extention

In [622]:
# Using requests.get to get the webpage in html format in a variable called page
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [623]:
# BeautifulSoup package will parse everything in an elegant way that was saved in page variable
soup = BeautifulSoup(page.content, 'html.parser')

In [624]:
# Finding the table from the parsed webpage. Developer tools in browser can help in finding the tag for table, row and individual element
html_table=soup.find('table')
# This command only works when there is only one table in the webpage

In [625]:
#Adding all rows from the table in the list
all_rows=html_table.find_all('tr')

In [626]:
#creating an empty list
final_data_list=[]

In [627]:
#Enumerate gives each row with numbers starting from 0
for pos,rows in enumerate(all_rows):
    # Creating a list of colums from individual elemets. We would only like text from the element in column hence we use the get_text(strip=True)

    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')]
    #Adding the column list to the final data list for each row number
    final_data_list.append(col_list)

In [628]:
#Pushing the final list to a Data frame
#Removing columns with na
#Renaming columns from 0: state and so on....
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state', 1:'cases', 2:'changes', 3:'cases_per_100k', 4:'fatal', 5:'comment'})

In [629]:
#Just checking the output of entered data in the Data Frame
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,3.815.110,0,51.576,"464,5",16.269
3,Bayern,5.098.273,0,70.557,"537,0",24.329
4,Berlin,1.092.531,0,15.239,"415,9",4.639
5,Branden­burg,816.462,0,9.532,"376,6",5.711
6,Bremen,213.932,682,5.229,"768,8",786


#Rest API calls
* We use German Corona Hub
* Got to data explorer
* Use data (possibility of downloading as csv file)
* Use API Explorer (it provides a Query URL)

In [630]:
#Using the Query URL to collect data in a variable. The result is in json format
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronafälle_in_den_Bundesländern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [631]:
#Creating json object from the variable data. Use json library in python
json_object=json.loads(data.content)

In [632]:
#Checking type of the data requested from the query URL
type(json_object)

dict

In [633]:
#Checking the keys of the imported data
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'geometryType', 'spatialReference', 'fields', 'features'])

In [634]:
#Creating an empty list to add the relevant data
full_list=[]
#Enumerate to go through all the features as numbered 0...
for pos,each_dict in enumerate(json_object['features']):
    #Adding the attributes for each feature to the full list
    full_list.append(each_dict['attributes'])

In [635]:
#pushing full list into a data frame
pd_full_list=pd.DataFrame(full_list)
#checking the data added in the data frame using API call
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death,cases7_bl_per_100k,cases7_bl,death7_bl,cases7_bl_per_100k_txt,AdmUnitId
0,1,01,Schleswig-Holstein,Land,2910875,15,814840,1656280800000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,27992.957444,4.573731e+10,2.881496e+06,2594,850.053678,24744,0,"850,1",1
1,2,02,Hamburg,Freie und Hansestadt,1852478,6,623752,1656280800000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,33671.223086,2.089396e+09,4.188002e+05,2717,611.937092,11336,5,"611,9",2
2,3,03,Niedersachsen,Land,8003421,9,2574734,1656280800000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,32170.418125,1.299836e+11,4.008988e+06,9614,825.746890,66088,4,"825,7",3
3,4,04,Bremen,Freie Hansestadt,680130,5,213932,1656280800000,04,4132268b-54de-4327-ac1e-760e915112f1,31454.574861,1.119157e+09,3.357177e+05,786,768.823607,5229,3,"768,8",4
4,5,05,Nordrhein-Westfalen,Land,17925570,10,5646024,1656280800000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,31497.040261,8.782936e+10,2.648673e+06,25694,707.163008,126763,12,"707,2",5


In [636]:
#Saving the created dataframe as a csv file in given directory
pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')

In [637]:
pd_full_list.shape[0]

16

#API access via REST service, e.g USA data

example of REST conform interface(attention registration mandatory)

REST: Representational State Transfer is a software architecture style which ensures interoperability between client/server communications

The URL from a REST API is divided as follows.
Operation   Server  Resource    Parameter

Registration on RapidAPI required. Subscribe with free tier to this specific API.

API available on GitHub of Smartable AI/developer-programs/

URL=https://rapidapi.com/SmartableAI/api/coronavirus-smartable/


In [638]:
#Select Python(Requests) in code snippets and then simply copy the code and paste it for usage:
url = "https://coronavirus-smartable.p.rapidapi.com/stats/v1/US/"
#Definition of headers. We have to define headers
headers = {
	"X-RapidAPI-Key": "021238aa51mshaba6f6f1d09d694p17f56djsn47111a01694b",
	"X-RapidAPI-Host": "coronavirus-smartable.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

print(response)

<Response [200]>


In [639]:
#Creating a dictonary from the json file requested from the REST API
US_dict=json.loads(response.content)
with open('../data/raw/RapidAPI/US_data.json', 'w') as outfile:
    json.dump(US_dict, outfile, indent=2)

In [640]:
print(json.dumps(US_dict, indent=2))

{
  "location": {
    "long": -95.712891,
    "countryOrRegion": "United States",
    "provinceOrState": null,
    "county": null,
    "isoCode": "US",
    "lat": 37.09024
  },
  "updatedDateTime": "2021-02-26T06:49:50.5325477Z",
  "stats": {
    "totalConfirmedCases": 28089251,
    "newlyConfirmedCases": 73964,
    "totalDeaths": 514334,
    "newDeaths": 2089,
    "totalRecoveredCases": 2796278,
    "newlyRecoveredCases": 0,
    "history": [
      {
        "date": "2020-01-22T00:00:00",
        "confirmed": 1,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-23T00:00:00",
        "confirmed": 1,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-24T00:00:00",
        "confirmed": 2,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-25T00:00:00",
        "confirmed": 2,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-26T00:00:00",
        "c

# Individual States US

In [641]:
US_dict['stats']['breakdowns'][0]

{'location': {'long': 144.793731,
  'countryOrRegion': 'United States',
  'provinceOrState': 'Guam',
  'county': None,
  'isoCode': None,
  'lat': 13.444304},
 'totalConfirmedCases': 32,
 'newlyConfirmedCases': 0,
 'totalDeaths': 1,
 'newDeaths': 0,
 'totalRecoveredCases': 0,
 'newlyRecoveredCases': 0}

In [642]:
full_list_US_country=[]
for pos,each_dict in enumerate(US_dict['stats']['breakdowns'][:]):
    flatten_dict=each_dict['location']
    flatten_dict.update(dict(list(US_dict['stats']['breakdowns'][pos].items())[1:7])
    )
    full_list_US_country.append(flatten_dict)

In [643]:
pd.DataFrame(full_list_US_country).to_csv('../data/raw/RapidAPI/full_list_US_country.csv', sep=';',index=False)

# Goals
* We would like to understand the data quality
* everything should be automated as much as possible: how many clicks do we need to exeute the full pipeline

# Constraints
Each notebook should be cleaned and ready for full execution.

Cleaning means no errors should be left behind, check for all the variable names etc.

Ready for execution means that once 'Run all' is pressed, all the code should run without error.